In [ ]:
%pip install -q apache-beam[interactive] --no-warn-conflicts

In [ ]:
%pip install google-cloud

In [ ]:
%pip install google-cloud-pubsub

In [ ]:
!git clone https://github.com/technoavengers/Apache_Beam.git

## **[Google PubSub Processor Without Windows](https://)** 
**Streaming Pipeline Without Windows**

* Reading from Movies Subscriptions (Movies topic)
* Filter movies that belongs to "Comedy Genre"
* Writing to Comedy topic

In [ ]:
import apache_beam as beam
from apache_beam.options.pipeline_options import PipelineOptions,StandardOptions
import os
from apache_beam import window

serviceAccount = '/content/sonorous-key-333217-def74b65ba97.json'
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]= serviceAccount

input_subscription = 'projects/sonorous-key-333217/subscriptions/movies-sub'

options= PipelineOptions()
options.view_as(StandardOptions).streaming= True

p = beam.Pipeline(options=options)

comedy_movies = 'projects/sonorous-key-333217/topics/comedy_movies'

pubsub_pipeline = (
    p
    | 'Read from pubsub topic' >> beam.io.ReadFromPubSub(subscription= input_subscription)
    | 'Split the records by comma' >> beam.Map(lambda row: row.decode("utf-8").split(','))
    | 'Filter movies with comedy genre' >> beam.Filter(lambda row: ("Comedy" in row[2]))
    | 'Converting to byte String' >> beam.Map(lambda row: (''.join(row).encode('utf-8')) )
    | 'Publish to output topic' >> beam.io.WriteToPubSub(comedy_movies)
)
result = p.run()
result.wait_until_finish()

## **[Google PubSub Processor With Tumbling Window](https://)** 
**Streaming Pipeline With Windows**

* Reading from Ratings Subscriptions (Ratings topic)
* Create Fixed side windows of 10s and count total ratings per key in each window

In [ ]:
import apache_beam as beam
from apache_beam.options.pipeline_options import PipelineOptions,StandardOptions
import os
from apache_beam import window
from apache_beam.transforms.combiners import Count
from apache_beam.runners.interactive.interactive_runner import InteractiveRunner
import apache_beam.runners.interactive.interactive_beam as ib


serviceAccount = '/content/sonorous-key-333217-def74b65ba97.json'
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]= serviceAccount

input_subscription = 'projects/sonorous-key-333217/subscriptions/ratings-sub'

options= PipelineOptions()
options.view_as(StandardOptions).streaming= True

p = beam.Pipeline(InteractiveRunner(),options=options)

ratings_count  = 'projects/sonorous-key-333217/topics/ratings_count'

pubsub_pipeline = (
    p
    | 'Read from pubsub topic' >> beam.io.ReadFromPubSub(subscription= input_subscription)
    | 'Split the records by comma' >> beam.Map(lambda row: row.decode("utf-8").split(','))
    | 'Form Key Value Pair' >> beam.Map(lambda rating: (rating[2],1))
    | 'Window' >> beam.WindowInto(window.FixedWindows(10))
    | 'Count the ratings' >> Count.PerKey()
    | 'Converting to byte String' >> beam.Map(lambda rating: (''.join(rating[0]+">>>"+str(rating[1])).encode('utf-8')) )
)

ib.show(pubsub_pipeline, include_window_info=True,visualize_data=True, n=50, duration=120)

## **[Google PubSub Processor With Sliding Window](https://)** 
**Streaming Pipeline With Windows**

* Reading from Ratings Subscriptions (Ratings topic)
* Create Fixed side windows of 20s and sliding interval of 5s and count total ratings per key in each window

In [ ]:
import apache_beam as beam
from apache_beam.options.pipeline_options import PipelineOptions,StandardOptions
import os
from apache_beam import window
from apache_beam.transforms.combiners import Count
from apache_beam.runners.interactive.interactive_runner import InteractiveRunner
import apache_beam.runners.interactive.interactive_beam as ib


serviceAccount = '/content/sonorous-key-333217-def74b65ba97.json'
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]= serviceAccount

input_subscription = 'projects/sonorous-key-333217/subscriptions/ratings-sub'

options= PipelineOptions()
options.view_as(StandardOptions).streaming= True

p = beam.Pipeline(InteractiveRunner(),options=options)

ratings_count  = 'projects/sonorous-key-333217/topics/ratings_count'

pubsub_pipeline = (
    p
    | 'Read from pubsub topic' >> beam.io.ReadFromPubSub(subscription= input_subscription)
    | 'Split the records by comma' >> beam.Map(lambda row: row.decode("utf-8").split(','))
    | 'Form Key Value Pair' >> beam.Map(lambda rating: (rating[2],1))
    | 'Window' >> beam.WindowInto(window.SlidingWindows(20,5))
    | 'Count the ratings' >> Count.PerKey()
    | 'Converting to byte String' >> beam.Map(lambda rating: (''.join(rating[0]+">>>"+str(rating[1])).encode('utf-8')) )
)

ib.show(pubsub_pipeline, include_window_info=True,visualize_data=True, n=50, duration=120)

## **[Google PubSub Processor With Session Window](https://)** 
**Streaming Pipeline With Windows**

* Reading from Ratings Subscriptions (Ratings topic)
* Create Session windows for max of 10s and count total ratings per key in each window

In [ ]:
import apache_beam as beam
from apache_beam.options.pipeline_options import PipelineOptions,StandardOptions
import os
from apache_beam import window
from apache_beam.transforms.combiners import Count
from apache_beam.runners.interactive.interactive_runner import InteractiveRunner
import apache_beam.runners.interactive.interactive_beam as ib


serviceAccount = '/content/sonorous-key-333217-def74b65ba97.json'
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]= serviceAccount

input_subscription = 'projects/sonorous-key-333217/subscriptions/ratings-sub'

options= PipelineOptions()
options.view_as(StandardOptions).streaming= True

p = beam.Pipeline(InteractiveRunner(),options=options)

ratings_count  = 'projects/sonorous-key-333217/topics/ratings_count'

pubsub_pipeline = (
    p
    | 'Read from pubsub topic' >> beam.io.ReadFromPubSub(subscription= input_subscription)
    | 'Split the records by comma' >> beam.Map(lambda row: row.decode("utf-8").split(','))
    | 'Form Key Value Pair' >> beam.Map(lambda rating: (rating[2],1))
    | 'Window' >> beam.WindowInto(window.Sessions(10))
    | 'Count the ratings' >> Count.PerKey()
    | 'Converting to byte String' >> beam.Map(lambda rating: (''.join(rating[0]+">>>"+str(rating[1])).encode('utf-8')) )
)

ib.show(pubsub_pipeline, include_window_info=True,visualize_data=True, n=50, duration=120)

## **[Google PubSub Processor With Session Window](https://)** 
**Streaming Pipeline With Windows**

* Reading from Ratings Subscriptions (Ratings topic)
* Create Session windows of 10 elements and count total ratings per key in each window

In [ ]:
import apache_beam as beam
from apache_beam.options.pipeline_options import PipelineOptions,StandardOptions
import os
from apache_beam import window
from apache_beam.runners.interactive.interactive_runner import InteractiveRunner
import apache_beam.runners.interactive.interactive_beam as ib

serviceAccount = '/content/sonorous-key-333217-def74b65ba97.json'
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]= serviceAccount

input_subscription = 'projects/sonorous-key-333217/subscriptions/movies-sub'

options= PipelineOptions()
options.view_as(StandardOptions).streaming= True

p = beam.Pipeline(InteractiveRunner(),options=options)

pubsub_pipeline = (
    p
    | 'Read from pubsub topic' >> beam.io.ReadFromPubSub(subscription= input_subscription)
    | 'Split the records by comma' >> beam.Map(lambda row: row.decode("utf-8").split(','))
    | 'Form Key Value Pair' >> beam.Map(lambda movies: (movies[2],1))
    | 'Window' >> beam.WindowInto(window.Sessions(5))
    | 'Count the ratings' >> Count.PerKey()
    | 'Converting to byte String' >> beam.Map(lambda genre: (''.join(genre[0]+">>>"+str(genre[1])).encode('utf-8')) )
)

ib.show(pubsub_pipeline, include_window_info=True,visualize_data=True, n=50, duration=120)

**Data Driven Triggers**

In [ ]:
import os
import apache_beam as beam
from apache_beam.options.pipeline_options import PipelineOptions,StandardOptions
from apache_beam import window
from apache_beam.transforms.combiners import Count
from apache_beam.transforms.trigger import AfterWatermark,AfterProcessingTime,AccumulationMode,AfterCount,Repeatedly,AfterAny
from apache_beam.runners.interactive.interactive_runner import InteractiveRunner
import apache_beam.runners.interactive.interactive_beam as ib


serviceAccount = '/content/sonorous-key-333217-def74b65ba97.json'
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]= serviceAccount

input_subscription = 'projects/sonorous-key-333217/subscriptions/ratings-sub'

options= PipelineOptions()
options.view_as(StandardOptions).streaming= True

p = beam.Pipeline(InteractiveRunner(),options=options)

pubsub_pipeline = (
    p
    | 'Read from pubsub topic' >> beam.io.ReadFromPubSub(subscription= input_subscription)
    | 'Split the records by comma' >> beam.Map(lambda row: row.decode("utf-8").split(','))
    | 'Form Key Value Pair' >> beam.Map(lambda row: (row[2],1))
    | 'Window' >> beam.WindowInto(window.FixedWindows(10),trigger=Repeatedly(AfterAny(AfterProcessingTime(5),AfterCount(10))),accumulation_mode=AccumulationMode.DISCARDING)
    | 'Count the ratings' >> Count.PerKey()
    | 'Converting to byte String' >> beam.Map(lambda genre: (''.join(genre[0]+">>>"+str(genre[1])).encode('utf-8'))
))

ib.show(pubsub_pipeline, include_window_info=True,visualize_data=True, n=50, duration=120)